In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
from sklearn.preprocessing import LabelEncoder
import joblib

In [2]:
data=pd.read_csv("twitter_sentiment.csv")
data.sample(5)

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
66712,7023,johnson&johnson,Neutral,<unk>&Johnson to stop making infant powder in ...
9930,12910,Xbox(Xseries),Negative,NaN
41193,1469,Battlefield,Negative,Disrespect to Call... of Duty.
48604,5943,HomeDepot,Negative,Man fuck up depot
21264,4041,CS-GO,Neutral,NaN


In [3]:
data=data[['Positive','im getting on borderlands and i will murder you all ,']].reset_index(drop=True)
data.head()

,Positive,"im getting on borderlands and i will murder you all ,"
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [4]:
data.columns=['sentiment','text']
data.head()

,sentiment,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [5]:
data.shape

(75681, 2)

In [6]:
# checking nan values
data.isna().sum()

sentiment      0
text         686
dtype: int64

In [7]:
#checking and dropping duplicates
data.duplicated().sum()
data.drop_duplicates(keep='first',inplace=True)
data.shape

(70253, 2)

In [8]:
data.dropna(subset=['text'],axis=0,inplace=True)
data.shape

(70249, 2)

In [54]:
lencoder=LabelEncoder()
data['sentiment']=lencoder.fit_transform(data['sentiment'])
joblib.dump(lencoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [10]:
lencoder.inverse_transform([0,1,2,3])

array(['Irrelevant', 'Negative', 'Neutral', 'Positive'], dtype=object)

In [11]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def text_process(text):
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    
    # Convert tokens to lowercase
    tokens = [word.lower() for word in tokens]
    
    # Convert list of tokens to a single string
    text = ' '.join(tokens)
    
    # --- helper sub‑functions ---
    def remove_html_tags(text):
        return re.sub(r'<.*?>', '', text)
    
    def remove_stopwords(text):
        words = [word for word in text.split() if word.lower() not in stop_words]
        return " ".join(words)
    
    def clean_url(text):
        text = re.sub(
            r"((https:|http|ftp)?(:\/\/)?(www\.)?)?"
            r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\."
            r"[a-zA-Z0-9()]{1,6}\b"
            r"([-a-zA-Z0-9()@:%_\+.~#?&\/\/=]*)",
            ' ',
            text
        )
        return re.sub(r'/', ' / ', text)
    
    def clean_punctuations(text):
        return text.translate(str.maketrans('', '', string.punctuation))
    
    def clean_repeating_char(text):
        return re.sub(r"(.)\1\1+", r"\1\1", text)
    
    def clean_numbers(text):
        return re.sub(r'[0-9]+', '', text)
    
    def remove_hashtag(text):
        return re.sub(r'#([\w\d]+)', ' ', text)          # raw string
    
    def clean_username(text):
        return re.sub(r'@[^\s]+', ' ', text)             # raw string
    
    def clean_non_ascii(text):
        return text.encode("ascii", "ignore").decode()
    
    def remove_images(tweet):
        cleaned_tweet = re.sub(r"pic\.twitter\.com/\S+", '', tweet)
        cleaned_tweet = re.sub(r"\w+(\.png|\.jpg|\.gif|\.jpeg)", " ", cleaned_tweet)  # raw string
        return cleaned_tweet
    
    def lemmatize_words(text):
        return " ".join(lemmatizer.lemmatize(word) for word in text.split())
    
    # --- apply all preprocessing steps ---
    text = remove_html_tags(text)
    text = remove_stopwords(text)
    text = clean_url(text)
    text = clean_punctuations(text)
    text = clean_repeating_char(text)
    text = clean_numbers(text)
    text = remove_hashtag(text)
    text = clean_username(text)
    text = clean_non_ascii(text)
    text = remove_images(text)
    text = lemmatize_words(text)
    
    return text

[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
data['text'] = data['text'].apply(text_process)
data.sample(10)

,sentiment,text
31123,1,im ded tired
68048,2,cyberpunk deferred development september th
68903,0,congrats team idsoftware winning game year
25064,1,mistake chen nt participate song please correc...
18737,1,imagine trying play new raid classic server ca...
43123,0,steal ouoxkitq
43154,1,dear pubg doesnt show care ban per week prof h...
29828,2,raqula best timed bombardment ever could clutc...
9600,2,xbox series x preorders debacle far rhyqd
4408,2,already cheating blackopscoldwar well come can...


In [13]:
data.to_csv("cleaned_twitter_sentiment.csv",index=False)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(data['text'])  # assuming data['text'] is cleaned

vocab_size = len(vectorizer.vocabulary_)
print("Vocabulary Size:", vocab_size)


Vocabulary Size: 28268
